<a href="https://colab.research.google.com/github/Nandhaceg7/trail/blob/main/logistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import gradio as gr
import numpy as np
from ortools.constraint_solver import routing_enums_pb2, pywrapcp

# 📌 Sample Distance Matrix (Replace with real distances)
DISTANCE_MATRIX = [
    [0, 10, 20, 30, 25],  # From Location 0
    [10, 0, 15, 35, 40],  # From Location 1
    [20, 15, 0, 25, 30],  # From Location 2
    [30, 35, 25, 0, 15],  # From Location 3
    [25, 40, 30, 15, 0]   # From Location 4
]

# 🚛 Route Optimization Function
def optimize_route(num_locations):
    if num_locations < 2 or num_locations > len(DISTANCE_MATRIX):
        return "❌ Please select at least 2 and at most 5 locations."

    # Create Data Model
    def create_data_model():
        return {
            'distance_matrix': [row[:num_locations] for row in DISTANCE_MATRIX[:num_locations]],
            'num_vehicles': 1,
            'depot': 0
        }

    data = create_data_model()

    # Create Routing Model
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    # Define Distance Callback
    def distance_callback(from_index, to_index):
        return data['distance_matrix'][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Set Search Parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Solve the Problem
    solution = routing.SolveWithParameters(search_parameters)

    # Extract Optimized Route
    if solution:
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        return f"🚚 Optimized Route: {route}"
    else:
        return "⚠ No optimal route found."

# 🌟 Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("<h1 style='text-align: center; color: #4CAF50;'>🚚 AI-Powered Route Optimization</h1>")
    gr.Markdown("<p style='text-align: center;'>Select the number of locations to optimize the delivery route.</p>")

    num_locations = gr.Slider(2, 5, value=3, step=1, label="📍 Number of Locations")
    predict_button = gr.Button("🚛 Find Best Route", variant="primary")
    output = gr.Markdown("")

    predict_button.click(fn=optimize_route, inputs=num_locations, outputs=output)

# Launch the App
app.launch()


ModuleNotFoundError: No module named 'ortools'